In [1]:
import pandas as pd
from tqdm import tqdm

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

client_id= "d8df752143c842238c774a4551d21546"
client_secret= "de7220a3104a43a0aed809594a5d92b1"

# Import data

In [2]:
pitchfork_df = pd.read_csv('pitchfork_reviews.tsv', sep=('\t'))

In [3]:
scrap_df = pitchfork_df[['title', 'artist']].drop_duplicates()

In [4]:
artist_df = scrap_df['artist'].drop_duplicates()

In [5]:
album_df = scrap_df['title'].drop_duplicates()

# Get artist URI for artist name

In [6]:
def initiate_connection(client_id, client_secret):
    
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API
    
    return sp

def get_artist_uri(sp, name):
    
    result = sp.search(name) #search query
    
    try:
        artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
        return artist_uri
        
    except Exception:
        print(result)
        return None

In [7]:
sp = initiate_connection(client_id, client_secret)

#artist_uri = get_artist_uri(sp, "Mystery Jets")

In [8]:
#art_uri_list = []

In [9]:
#for name in list(artist_df):
#    try:
#        art_uri_list.append([name, get_artist_uri(sp, name)])
#    except Exception:
#        break

In [10]:
#art_uri_df = pd.DataFrame(art_uri_list, columns=['artist', 'artist_uri'])

In [11]:
#art_uri_df.to_csv('art_uri.csv')

In [12]:
art_uri_df = pd.read_csv('art_uri.csv', index_col=False)

# Get album names and URI for each artist

In [13]:
scrap_artist_df = scrap_df.merge(art_uri_df)

In [14]:
#album_uri_list = []

In [15]:
def get_all_artist_albums(sp, artist, artist_uri):

    alb = []
    
    #Pull all of the artist's albums
    sp_albums = sp.artist_albums(artist_uri, album_type='album')
    
    try:
    
        #Store artist's albums' names' and uris in separate lists
        for i in range(len(sp_albums['items'])):
            alb.append([artist, artist_uri, sp_albums['items'][i]['name'], sp_albums['items'][i]['uri']])
            
        return alb
            
    except Exception:
        print(sp_albums)
        pass

In [16]:
#for row in tqdm(art_uri_df.itertuples(index=False)):
#    try:
#        album_uri_list.extend(get_all_artist_albums(sp, row[0], row[1]))
#    except Exception:
#        pass

In [17]:
#album_uri_df = pd.DataFrame(album_uri_list, columns=['artist', 'artist_uri', 'name', 'album_uri'])

In [18]:
#album_uri_df.to_csv('album_uri.csv')

In [19]:
album_uri_df = pd.read_csv('album_uri.csv', index_col=False)

In [20]:
album_uri_df = album_uri_df.rename(columns={'name':'title'})

# Clean album names, join

In [21]:
import jellyfish

In [22]:
common = scrap_df.merge(album_uri_df, on=['artist', 'title'])

In [23]:
common['not_common'] = False

In [24]:
filter_not_common = scrap_df.merge(common[['title', 'artist', 'not_common']].drop_duplicates(), 
                               how='left')['not_common'].fillna(True)

In [25]:
to_join_df = album_uri_df.merge(scrap_df.reset_index().loc[filter_not_common], on='artist')

In [26]:
to_join_df['similarity'] = to_join_df.apply(lambda row: jellyfish.jaro_winkler(str(row['title_x']), str(row['title_y'])), axis=1)

In [27]:
not_common = to_join_df.sort_values('similarity', ascending=False).groupby(['artist', 'title_y']).head(1)

In [28]:
not_common = not_common[not_common.similarity > 0.8]

In [29]:
not_common = not_common.rename(columns={'title_y':'title'})

In [30]:
common_clean = common[['title', 'artist', 'artist_uri', 'album_uri']].groupby(['artist', 'title']).head(1)

In [31]:
common_clean = common_clean.append(not_common[['title', 'artist', 'artist_uri', 'album_uri']])

In [32]:
common_clean.to_csv('common_clean.csv')

# Get track information, aggregate and link to album

In [33]:
def get_album_features(sp, album_uri):
    
    tracks = sp.album_tracks(album_uri)
    
    acousticness = []
    danceability = []
    energy = []
    instrumental = []
    liveness = []
    loudness = []
    speechiness = []
    tempo = []
    valence = []
    
    popularity = []
    
    for i in range(len(tracks['items'])):
        
        track_uri = tracks['items'][i]['uri']
        
        features = sp.audio_features(track_uri)
       
        acousticness.append(features[0]['acousticness'])
        danceability.append(features[0]['danceability'])
        energy.append(features[0]['energy'])
        instrumental.append(features[0]['instrumentalness'])
        liveness.append(features[0]['liveness'])
        loudness.append(features[0]['loudness'])
        speechiness.append(features[0]['speechiness'])
        tempo.append(features[0]['tempo'])
        valence.append(features[0]['valence'])
        
        pop = sp.track(track_uri)
        popularity.append(pop['popularity'])
        
    return {'album_uri': album_uri,
            'acousticness': np.mean(acousticness),
            'danceability': np.mean(danceability),
            'energy': np.mean(energy),
            'instrumental': np.mean(instrumental),
            'liveness': np.mean(liveness),
            'loudness': np.mean(loudness),
            'speechiness': np.mean(speechiness),
            'tempo': np.mean(tempo),
            'valence': np.mean(valence),
            'popularity': np.mean(popularity)}

In [34]:
album_features = []

In [35]:
import time
import numpy as np


sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0

for row in tqdm(common_clean.iloc[5000:5500].itertuples(index=False)):

    try:
        album_features.append(get_album_features(sp, row[3]))
        
        request_count+=1
        
        if request_count % 10 == 0:
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            
    except Exception:
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        pass

58it [02:12,  1.90s/it]

retrying ...1secs
retrying ...1secs


78it [03:03,  2.11s/it]

retrying ...1secs
retrying ...1secs


99it [03:50,  1.78s/it]

...

437it [17:42,  2.27s/it]

retrying ...1secs
retrying ...1secs


448it [18:22,  4.09s/it]

retrying ...1secs
retrying ...1secs


500it [20:38,  2.48s/it]


In [92]:
pd.DataFrame(album_features).to_csv('album_features_4999_to_5499.csv')

In [88]:
len(album_features)

3800

In [58]:
common_clean.reset_index(drop=True).iloc[25:300]

,title,artist,artist_uri,album_uri
25,333,hour of 13,spotify:artist:5xgm2Kh3oTlcFEKdHovNKA,spotify:album:5OZy0tvQ9dpVHija0WKef1
26,sorry,white lung,spotify:artist:2iT2Fmot4VzWgdOTgp3j9M,spotify:album:78WM9cuBpyk0MvI2lcEQIa
27,the plot against common sense,future of the left,spotify:artist:4waRXR1luGWkwHOPZUjz2W,spotify:album:07t1mMckO61tMce67UI2VH
28,1966,karen dalton,spotify:artist:5O5V29YvM6AzAQ0rNt59fy,spotify:album:1K7tpapZGd6Ax1YA2vXNEA
29,flumina,"fennesz, sakamoto",spotify:artist:2DoQBgPsB9AdmWpIa2hUSz,spotify:album:1w3nnQ2obGiUAsNgqv0RE1
...,...,...,...,...
295,flaamingos,flaamingos,spotify:artist:13m29DGHBxpcVYS734KCvx,spotify:album:4uaciprpeOrZc0rrRlue0p
296,belladonna,daniel lanois,spotify:artist:5S0AJvE9NB1kGrXRfYh690,spotify:album:05tAz0b9gHjjlpaIOxNvfy
297,nootropics,lower dens,spotify:artist:5uTJLo3c4ZHpH8oEwGxYUN,spotify:album:5ovZHENU63vV3rSYcTYE1U
298,"celebration, florida",the felice brothers,spotify:artist:4Ajgo7nAsTzjSFymIfBjZ1,spotify:album:70QqtUN1grBIsT0mQhxEUk
